<a href="https://colab.research.google.com/github/KoMurase/Learn_NLP/blob/master/torch_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install janome

     |████████████████████████████████| 21.5MB 1.4MB/s 


In [0]:
from janome.tokenizer import Tokenizer 

j_t = Tokenizer()

def tokenizer_janome(text):
  return [tok for tok in j_t.tokenize(text, wakati = True)]

In [0]:
#前処理用関数
import re 

def preprocessing_text(text):
  #英文字の小文字化
  #output = output.lower()

  # 改行コード、全角、半角の削除　
  text = re.sub('\r', '', text)  
  text = re.sub('\n', '', text)  
  text = re.sub('　', '', text)
  text = re.sub(' ', '', text)

  #数字文字の一律化
  text = re.sub(r'[0-9 0-9]', '0', text)

  return text

In [0]:
def tokenizer_with_preprocessing(text):
  text = preprocessing_text(text)
  ret = tokenizer_janome(text)

  return ret 

In [5]:
text = "最近、　コロナウイルスで 自粛して家にいる。今は19時33分だ。"
print(tokenizer_with_preprocessing(text))

['最近', '、', 'コロナ', 'ウイルス', 'で', '自粛', 'し', 'て', '家', 'に', 'いる', '。', '今', 'は', '00', '時', '00', '分', 'だ', '。']


In [0]:
path = '/content/drive/My Drive/pytorch_nlp/'

In [7]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import torchtext 
#文章データの読み込み
#torch.data.Fieldの引数で実行する前処理が指定できる

max_length = 25 

TEXT = torchtext.data.Field(
    sequential = True, #データの長さが可変かどうか。ラベルデータにはFalseを適用する
    tokenize = tokenizer_with_preprocessing, #上で定義したやつ  ### ←ここ強化する
    use_vocab = True, #単語をボキャブラリーに追加するかどうか 
    lower = True, #アルファベットを小文字にするかどうか 
    include_lengths = True,  #文章の単語数のデータを保持するかどうか 
    batch_first = True, #ミニバッチの次元を先頭に保持するかどうか 
    fix_length = max_length  #全部の文章を指定した長さと同じになるように paddingする
)

LABEL = torchtext.data.Field(
    sequential = False, 
    use_vocab = False, 
)

In [0]:
path = '/content/drive/My Drive/pytorch_nlp/'

In [0]:
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path = path, train = 'sample1.csv', 
    validation = 'sample2.csv',
    test = 'sample3.csv',
    format = 'csv',
    skip_header=True,   ###　headerを読み込まない処理
    fields = [('Label', LABEL),('Text', TEXT)]
)

In [15]:
train_ds

In [16]:
#動作確認
print('訓練データの数', len(train_ds))
print('訓練データ[1]', vars(train_ds))

訓練データの数 10
訓練データ[1] {'examples': [<torchtext.data.example.Example object at 0x7fc94c76bf60>, <torchtext.data.example.Example object at 0x7fc94c76bf98>, <torchtext.data.example.Example object at 0x7fc94c76bfd0>, <torchtext.data.example.Example object at 0x7fc94c6cc0f0>, <torchtext.data.example.Example object at 0x7fc94c6cc1d0>, <torchtext.data.example.Example object at 0x7fc94c6cc048>, <torchtext.data.example.Example object at 0x7fc94c6cc160>, <torchtext.data.example.Example object at 0x7fc94c6cc630>, <torchtext.data.example.Example object at 0x7fc94c6cc240>, <torchtext.data.example.Example object at 0x7fc94c6cc828>], 'fields': {'Label': <torchtext.data.field.Field object at 0x7fc999e23e48>, 'Text': <torchtext.data.field.Field object at 0x7fc999e23e80>}}


In [17]:
print('訓練データ[1]', vars(train_ds[0])['Text'])

訓練データ[1] ['【', 'マイ', 'リスト', '】', 'カカカカ', '覚醒剤', 'https', '://', 't', '.', 'co', '/', '00', 'zy', '0', 'oa', '0', 'qs', '#', 'sm', '00000000']


In [18]:
print('訓練データ[1]', vars(train_ds[0])['Label'])

訓練データ[1] 0


In [19]:
#動作確認
print('訓練データの数', len(train_ds))
print('訓練データ[1]', vars(train_ds[1])['Text'])

訓練データの数 10
訓練データ[1] ['@', 'kenjikenji', '0000', 'せ', 'や', 'な', '落雷', '撃た', 'れ', 'たら', '全員', '張り付く', 'しか', 'なかっ', 'た', 'から', 'なぁ', 'なん', 'で', '速攻', 'で', '雷', '銅', '狙い', 'に', '行っ', 'た', 'わ']


In [0]:
TEXT.build_vocab(train_ds, min_freq=1)
#訓練データtrainの単語からmin_freqの以上の頻度の単語を使用して単語集を構築する


In [23]:
TEXT.vocab.freqs

Counter({'#': 1,
         '-': 2,
         '.': 3,
         '...': 1,
         '/': 4,
         '0': 6,
         '00': 3,
         '0000': 3,
         '00000000': 1,
         '://': 3,
         '@': 2,
         '[': 1,
         ']': 1,
         'co': 3,
         'etc': 1,
         'gw': 1,
         'https': 3,
         'i': 1,
         'it': 1,
         'jadevalerie': 1,
         'kenjikenji': 1,
         'livingbynumbers': 1,
         'ltejjx': 1,
         'mr': 1,
         'n': 1,
         'oa': 1,
         'p': 1,
         'qs': 1,
         'rdj': 1,
         'sm': 1,
         't': 4,
         'uiz': 1,
         'w': 1,
         'yt': 1,
         'zy': 1,
         '…': 1,
         '⇒「': 1,
         '♪': 1,
         '、': 11,
         '。': 15,
         '「': 3,
         '」': 4,
         '【': 3,
         '】': 3,
         '〜': 1,
         'あげ': 1,
         'あなた': 1,
         'あの': 1,
         'ある': 2,
         'い': 2,
         'いし': 2,
         'いる': 4,
         'いわし': 1,
         'うえ': 

In [24]:
TEXT.vocab.stoi #どの単語がどの単語IDに割り振られたのかを確認できる

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'#': 67,
             '-': 39,
             '.': 25,
             '...': 68,
             '/': 16,
             '0': 11,
             '00': 26,
             '0000': 27,
             '00000000': 69,
             '://': 28,
             '<pad>': 1,
             '<unk>': 0,
             '@': 40,
             '[': 70,
             ']': 71,
             'co': 29,
             'etc': 72,
             'gw': 73,
             'https': 30,
             'i': 74,
             'it': 75,
             'jadevalerie': 76,
             'kenjikenji': 77,
             'livingbynumbers': 78,
             'ltejjx': 79,
             'mr': 80,
             'n': 81,
             'oa': 82,
             'p': 83,
             'qs': 84,
             'rdj': 85,
             'sm': 86,
             't': 17,
             'uiz': 87,
             'w': 88,
             'yt': 89,
             'zy': 90,
             '…': 91,
             '⇒「': 92,
    

##DataLoaderの作成

In [0]:
train_dl = torchtext.data.Iterator(train_ds, batch_size=2, train=True)

val_dl = torchtext.data.Iterator(val_ds, batch_size=2, train=False, sort=False)

test_dl = torchtext.data.Iterator(test_ds, batch_size=2, train=False, sort=False)

In [27]:
batch = next(iter(val_dl))
print(batch.Text)


(tensor([[  0,  36,   0,   0,   0,   0,   0,   0,   0,   7, 162,  58,   0,   0,
           8,   0,  50,   5, 126,  91,   0,   7,   0,   6,  91],
        [  0,   0,   0,   3,   0,   3,   0,   3,   0,   8,   0,   0,   0,   0,
           0,   0,   4,   0,   3,   0,  14,   0,   0,   2,   0]]), tensor([25, 25]))


In [28]:
print(batch.Label)

tensor([0, 0])
